In [1]:
import pandas as pd
import numpy as np
import os
import sys
import importlib
import joblib

from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import lightgbm as lgb
from sklearn.metrics import roc_curve,roc_auc_score

import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

In [2]:
sys.path.append('/Users/hb26404/Desktop/内部模型/0建模脚本')
import myfuncs as mf
importlib.reload(mf)

(CVXPY) Jun 19 04:02:05 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.9.3963). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jun 19 04:02:05 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.9.3963). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')


<module 'myfuncs' from '/Users/hb26404/Desktop/内部模型/0建模脚本/myfuncs.py'>

In [3]:
left_fes = ['翡翠92',
 '明鉴92',
 '玲珑92',
 '琳琅93',
 'SC_C11010',
 '小满分V12PRO-互金版高定价2',
 '小满分消金定制版3',
 'Score_hk52',
 'Score_hy51',
 'Score_o34',
 'afc_yh_cus_dq_v2',
 'hello_v6',
 'ant_anti_insight_score_v6',
 '融安云联分-标准款',
 'yaosi_ty_2',
 'tuyuan_207',
 'jm_s1',
 '50002',
 'pudao_tengxun_external_dp_loan_score_v1',
 'pudao_tengxun_external_model_risk_v8_stongyong_score',
 'pudao_tengxun_external_dp_credit_score_v1',
 'youmeng_external_risk_score_y7_v1',
 'youmeng_external_risk_score_y7_v2',
 'youmeng_external_risk_score_y7_v3',
 'yidun_external_subscore1_v1',
 'yidun_external_subscore2_v1',
 'yidun_external_subscore3_v1',
 'yidun_external_subscore4_v1',
 'yidun_external_subscore5_v1',
 'yidun_external_subscore6_v1',
 'yidun_external_subscore7_v1',
 'yidun_external_subscore8_v1',
 'yidun_external_subscore9_v1',
 ]



left_fes_name= {
    
    # 信易搜相关
    "翡翠92": "xys_fc92", 
    "明鉴92": "xys_mj92", 
    "玲珑92": "xys_linglong92", 
    "琳琅93": "xys_linlang93", 
    # 电话邦相关
     "SC_C11010":'dhb_SC_C11010',
     
   #度小满
    
    "小满分V12PRO-互金版高定价2":'dxm_V12PRO-hjgdj2',
     "小满分消金定制版3":'dxm_xjdz3',
    
    # 恒指普惠相关
    "Score_hk52":'hengzhi_Score_hk52', 
    "Score_hy51":'hengzhi_Score_hy51',
    "Score_o34":'hengzhi_Score_o34',
   
    
    # 蚂蚁相关
    "afc_yh_cus_dq_v2":'mayi_afc_yh_cus_dq_v2', 
    "hello_v6":'mayi_hello_v6',
    "ant_anti_insight_score_v6":'ant_anti_insight_score_v6',
    
    # 百融相关
     "融安云联分-标准款":'bairong_yunlianfen',

     #避雷针
    "yaosi_ty_2":'blz_yaosi_ty_2', 
    'tuyuan_207':'blz_tuyuan_207',
    
    #极光
    "jm_s1":'jg_jm_s1', 

    # 火山相关
    "50002":'zj_50002',

    #腾讯
    "pudao_tengxun_external_dp_loan_score_v1":'pudao_tengxun_external_dp_loan_score_v1',
    "pudao_tengxun_external_model_risk_v8_stongyong_score":'pudao_tengxun_external_model_risk_v8_stongyong_score',
     "pudao_tengxun_external_dp_credit_score_v1":'pudao_tengxun_external_dp_credit_score_v1',

     #友盟
    "youmeng_external_risk_score_y7_v1":'youmeng_external_risk_score_y7_v1',
    "youmeng_external_risk_score_y7_v2":'youmeng_external_risk_score_y7_v2',
    "youmeng_external_risk_score_y7_v3":'youmeng_external_risk_score_y7_v3',
 
    
     #蚂蚁9个子分
       "yidun_external_subscore1_v1":'yidun_external_subscore1_v1',
       "yidun_external_subscore2_v1":'yidun_external_subscore2_v1',
       "yidun_external_subscore3_v1":'yidun_external_subscore3_v1',
       "yidun_external_subscore4_v1":'yidun_external_subscore4_v1',
       "yidun_external_subscore5_v1":'yidun_external_subscore5_v1',
       "yidun_external_subscore6_v1":'yidun_external_subscore6_v1',
       "yidun_external_subscore7_v1":'yidun_external_subscore7_v1',
       "yidun_external_subscore8_v1":'yidun_external_subscore8_v1',
       "yidun_external_subscore9_v1":'yidun_external_subscore9_v1',
       
}

fes_all = list(left_fes_name.values())

len(left_fes_name)

33

In [4]:
df_score = pd.read_csv('/Users/hb26404/Desktop/内部模型/c924项目A卡模型2505/2数据分析/merged_dfv3.csv')
# df_score['date'] = pd.to_datetime(df_score['apply_date'].astype('str')).astype('str')
# df_score['mon'] = df_score['date'].str[:7]
df_score = df_score[['mob_md5', 'apply_date']+left_fes]
df_score = df_score.rename(columns=left_fes_name)
df_score.shape

(300000, 35)

In [5]:
df_sam = pd.read_csv('/Users/hb26404/Desktop/内部模型/c924项目A卡模型2505/1样本及数据/外发样本/30W外发样本所有基本信息250617.csv')
df_sam.shape

(300000, 35)

In [6]:
df_sam.is_zy.value_counts()

is_zy
1    211235
0     88765
Name: count, dtype: int64

In [7]:
df_merge = df_sam.merge(df_score, how='inner', on=['mob_md5', 'apply_date'])
df_merge.shape

(300000, 68)

In [8]:
df_merge[fes_all] = df_merge[fes_all].replace([-999, -2, -1], [np.nan, np.nan, np.nan])
df_merge['mon'] = df_merge['date'].str[:7] 
df_merge.shape

(300000, 69)

In [9]:
label_list = ['label_fpd_7', 'label_fpd_15', 'label_fpd_30', 'label_2pd_7', 'label_2pd_15',
       'label_2pd_30', 'label_3pd_7', 'label_3pd_15', 'label_3pd_30',
       'label_4pd_15', 'label_4pd_30', 'label_5pd_30', 'label_6pd_30',]

df_sam_tmp = df_sam.copy()
df_sam_tmp['mon'] = df_sam_tmp['date'].str[:7]
df_sam_tmp[label_list] = df_sam_tmp[label_list].replace(0.5, 0)

df_sam_tmp.groupby(['mon'])[label_list].sum()

,label_fpd_7,label_fpd_15,label_fpd_30,label_2pd_7,label_2pd_15,label_2pd_30,label_3pd_7,label_3pd_15,label_3pd_30,label_4pd_15,label_4pd_30,label_5pd_30,label_6pd_30
mon,,,,,,,,,,,,,
2024-10,461.0,366.0,297.0,1111.0,912.0,763.0,1739.0,1432.0,1252.0,1912.0,1715.0,2221.0,2778.0
2024-11,425.0,340.0,280.0,1039.0,869.0,741.0,1624.0,1378.0,1221.0,1844.0,1676.0,2151.0,1499.0
2024-12,463.0,367.0,304.0,1093.0,913.0,802.0,1748.0,1479.0,1309.0,2113.0,1909.0,1164.0,0.0
2025-01,606.0,501.0,432.0,1448.0,1249.0,1094.0,2282.0,1974.0,1781.0,2598.0,1209.0,0.0,0.0
2025-02,778.0,662.0,563.0,1736.0,1512.0,1321.0,2705.0,2214.0,1144.0,130.0,0.0,0.0,0.0
2025-03,903.0,748.0,645.0,2183.0,1854.0,869.0,985.0,138.0,0.0,0.0,0.0,0.0,0.0
2025-04,356.0,301.0,170.0,333.0,47.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-05,66.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
cut_date = '2024-08-01'
oot_date = '2025-03-15'

cut_date1 = '2024-10-01'

y_label = 'label_2pd_15'

feas = fes_all+['feat_loan_risk_gender_from_idcard',  'feat_loan_risk_province_from_idcard', 'feat_loan_risk_age_from_idcard']

df_tt = df_merge[(df_merge.date<=oot_date)&(df_merge[y_label].isin([0,1]))]
# df_tt = df_merge[(df_merge.date>=cut_date) & (df_merge.date<=oot_date)&(df_merge[y_label].isin([0,1]))]
# df_tt = df_merge[(df_merge.date>=cut_date1)&(df_merge.date<='2025-01-31')&(df_merge[y_label].isin([0,1]))]

df_oot = df_merge[(df_merge.date>oot_date)&(df_merge[y_label].isin([0,1]))]
df_oot1 = df_merge[(df_merge.date>oot_date)&(df_merge[y_label].notnull())]
# df_oot = df_merge[(df_merge.date>=cut_date)& (df_merge.date<cut_date1) &(df_merge[y_label].isin([0,1]))]
# df_oot1 = df_merge[(df_merge.date>=cut_date)& (df_merge.date<cut_date1)&(df_merge[y_label].notnull())]
df_oot1[y_label] = df_oot1[y_label].replace(0.5,0)

df_oot1.shape

(27814, 69)

In [11]:
df_merge2 = df_merge.copy()
df_merge2[label_list] = df_merge2[label_list].replace(0.5, 0)

In [12]:
train, test=train_test_split(df_tt, test_size=0.2, stratify=df_tt[y_label], random_state=1234)
y_train = train[y_label]
y_test = test[y_label]
y_oot = df_oot[y_label]
y_oot1 = df_oot1[y_label]


samples_info = pd.DataFrame(index=['train','test', 'oot', 'oot1'])
samples_info['样本量'] = [y_train.count(),y_test.count(),y_oot.count(),y_oot1.count()]
samples_info['样本坏量'] = [y_train.sum(),y_test.sum(),y_oot.sum(),y_oot1.sum()]
samples_info['样本坏率'] = [y_train.mean(),y_test.mean(),y_oot.mean(),y_oot1.mean()]
samples_info

,样本量,样本坏量,样本坏率
train,161799,5106.0,0.031558
test,40450,1277.0,0.031570
oot,23843,973.0,0.040809
oot1,27814,973.0,0.034982


In [313]:
loss_rate = (train[feas].isnull()).mean()
loss_drop = loss_rate[loss_rate>0.9].index.tolist()
# print(loss_drop)
len(loss_drop)

0

In [314]:
unique_num = train[feas].nunique()
unique_drop = unique_num[unique_num<=1].index.tolist()
len(unique_drop)

0

In [315]:
cat_fes=[]
special_list = []

psi_df = pd.DataFrame()
psi_tt = {}
psi_to = {}
psi_to1 = {}
psi_to2 = {}
iv_res = {}
iv_bins = {}

psi_bad_df = pd.DataFrame()
psi_bad_tt = {}
psi_bad_to = {}
psi_bad_to1 = {}

for col in tqdm(feas):
    base_col = train[col]
    test_col = test[col]
    oot_col = df_oot[col]
    oot_new_col = df_merge[df_merge.date>='2025-03-01'][col]

    base_bad_col = train[train[y_label]==1][col]
    test_bad_col = test[test[y_label]==1][col]
    oot_bad_col = df_oot[df_oot[y_label]==1][col]

    if col in cat_fes:
        psi_tt[col],_ ,_= mf.calculate_psi(base_col, test_col, bins=5, min_sample=10, numerical=False)
        psi_to[col],_ ,_= mf.calculate_psi(base_col, oot_col, bins=5, min_sample=10, numerical=False)
        psi_to1[col],_ ,_= mf.calculate_psi(test_col, oot_col, bins=5, min_sample=10, numerical=False)
        psi_to2[col],_ ,_= mf.calculate_psi(base_col, oot_new_col, bins=5, min_sample=10, numerical=False)
    else:
        psi_tt[col],_ ,_= mf.calculate_psi(base_col, test_col, bins=5, min_sample=10, numerical=True)
        psi_to[col],_ ,_= mf.calculate_psi(base_col, oot_col, bins=5, min_sample=10, numerical=True)
        psi_to1[col],_ ,_= mf.calculate_psi(test_col, oot_col, bins=5, min_sample=10, numerical=True)
        psi_to2[col],_ ,_= mf.calculate_psi(base_col, oot_new_col, bins=5, min_sample=10, numerical=True)

        psi_bad_tt[col],_ ,_= mf.calculate_psi(base_bad_col, test_bad_col, bins=5, min_sample=10, numerical=True)
        psi_bad_to[col],_ ,_= mf.calculate_psi(base_bad_col, oot_bad_col, bins=5, min_sample=10, numerical=True)
        psi_bad_to1[col],_ ,_= mf.calculate_psi(test_bad_col, oot_bad_col, bins=5, min_sample=10, numerical=True)

    if loss_rate[col]==1 or unique_num[col]==0:
        iv_res[col], iv_bins[col] = 0, None
    else:
        if col in cat_fes:
            iv_res[col], iv_bins[col], _ = mf.calculate_iv(base_col, train[y_label], name=col, max_n_bins=5, special_codes=special_list, min_prebin_size=0.05, dtype='categorical')
        else:
            iv_res[col], iv_bins[col], _ = mf.calculate_iv(base_col, train[y_label], name=col, max_n_bins=5, special_codes=special_list, min_prebin_size=0.05, dtype='numerical')


psi_df['tt'] = pd.Series(psi_tt)
psi_df['to'] = pd.Series(psi_to)
psi_df['to1'] = pd.Series(psi_to1)
psi_df['to2'] = pd.Series(psi_to2)

psi_bad_df['tt'] = pd.Series(psi_bad_tt)
psi_bad_df['to'] = pd.Series(psi_bad_to)
psi_bad_df['to1'] = pd.Series(psi_bad_to1)

iv_res = pd.Series(iv_res)

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [00:06<00:00,  5.46it/s]


In [316]:
psi_drop = psi_df[(psi_df>0.1).any(axis=1)].index.tolist()
len(psi_drop)

1

In [317]:
psi_drop

['dhb_SC_C11010']

In [318]:
psi_df

,tt,to,to1,to2
xys_fc92,0.000040,0.009828,0.009889,0.006801
xys_mj92,0.000353,0.004857,0.003105,0.006113
xys_linglong92,0.000019,0.015675,0.016533,0.011702
xys_linlang93,0.000062,0.000952,0.001199,0.001269
dhb_SC_C11010,0.000057,0.414147,0.421301,0.170976
dxm_V12PRO-hjgdj2,0.000075,0.004129,0.004305,0.000495
dxm_xjdz3,0.000149,0.057464,0.054935,0.041359
hengzhi_Score_hk52,0.000114,0.015647,0.015611,0.013665
hengzhi_Score_hy51,0.000266,0.001692,0.000938,0.002048
hengzhi_Score_o34,0.000045,0.024038,0.023174,0.024916


In [319]:
psi_bad_drop = psi_bad_df[(psi_bad_df>0.05).any(axis=1)].index.tolist()
len(psi_bad_drop)

1

In [320]:
psi_bad_df

,tt,to,to1
xys_fc92,0.002238,0.011692,0.012210
xys_mj92,0.006346,0.005352,0.001312
xys_linglong92,0.008485,0.017151,0.021441
xys_linlang93,0.005626,0.012010,0.023805
dhb_SC_C11010,0.005580,0.340113,0.358416
dxm_V12PRO-hjgdj2,0.002012,0.015174,0.020807
dxm_xjdz3,0.002110,0.041299,0.040020
hengzhi_Score_hk52,0.002444,0.016476,0.010151
hengzhi_Score_hy51,0.001632,0.002054,0.005444
hengzhi_Score_o34,0.005357,0.012419,0.033737


In [321]:
iv_drop = iv_res[iv_res<0.00].index.tolist()
len(iv_drop)

0

In [322]:
iv_res

xys_fc92                                                0.121312
xys_mj92                                                0.093653
xys_linglong92                                          0.090842
xys_linlang93                                           0.075206
dhb_SC_C11010                                           0.051896
dxm_V12PRO-hjgdj2                                       0.135391
dxm_xjdz3                                               0.053112
hengzhi_Score_hk52                                      0.090447
hengzhi_Score_hy51                                      0.081468
hengzhi_Score_o34                                       0.093524
mayi_afc_yh_cus_dq_v2                                   0.124231
mayi_hello_v6                                           0.110693
ant_anti_insight_score_v6                               0.083654
bairong_yunlianfen                                      0.054468
blz_yaosi_ty_2                                          0.128127
blz_tuyuan_207           

In [323]:
fes_all

['xys_fc92',
 'xys_mj92',
 'xys_linglong92',
 'xys_linlang93',
 'dhb_SC_C11010',
 'dxm_V12PRO-hjgdj2',
 'dxm_xjdz3',
 'hengzhi_Score_hk52',
 'hengzhi_Score_hy51',
 'hengzhi_Score_o34',
 'mayi_afc_yh_cus_dq_v2',
 'mayi_hello_v6',
 'ant_anti_insight_score_v6',
 'bairong_yunlianfen',
 'blz_yaosi_ty_2',
 'blz_tuyuan_207',
 'jg_jm_s1',
 'zj_50002',
 'pudao_tengxun_external_dp_loan_score_v1',
 'pudao_tengxun_external_model_risk_v8_stongyong_score',
 'pudao_tengxun_external_dp_credit_score_v1',
 'youmeng_external_risk_score_y7_v1',
 'youmeng_external_risk_score_y7_v2',
 'youmeng_external_risk_score_y7_v3',
 'yidun_external_subscore1_v1',
 'yidun_external_subscore2_v1',
 'yidun_external_subscore3_v1',
 'yidun_external_subscore4_v1',
 'yidun_external_subscore5_v1',
 'yidun_external_subscore6_v1',
 'yidun_external_subscore7_v1',
 'yidun_external_subscore8_v1',
 'yidun_external_subscore9_v1']

In [324]:
# corr_tmp = train[fes_all].corr('spearman')

In [325]:
# corr_tmp.abs().to_csv('初选相关性_tmp.csv')

In [326]:
add_drop_fes = [ 
    'pudao_tengxun_external_dp_loan_score_v1',
    'pudao_tengxun_external_model_risk_v8_stongyong_score',
    'pudao_tengxun_external_dp_credit_score_v1',
    
    'youmeng_external_risk_score_y7_v1',
    'youmeng_external_risk_score_y7_v2',
    'youmeng_external_risk_score_y7_v3',

     'yidun_external_subscore1_v1',
    'yidun_external_subscore2_v1',
    'yidun_external_subscore3_v1',
    'yidun_external_subscore4_v1',
    'yidun_external_subscore5_v1',
    'yidun_external_subscore6_v1',
    'yidun_external_subscore7_v1',
    'yidun_external_subscore8_v1',
    'yidun_external_subscore9_v1',

    'jg_jm_s1',

    # 'mayi_afc_yh_cus_dq_v2',
    'ant_anti_insight_score_v6',

    'dxm_xjdz3',
    
    # 'xys_linglong92',
    'xys_linlang93',

    'hengzhi_Score_hy51',
    'hengzhi_Score_o34',
    'blz_tuyuan_207',
    'bairong_yunlianfen',

    'hengzhi_Score_hk52',

    # 'mayi_hello_v6',
]

corr_drop = ['xys_mj92', ]

In [327]:
feas1 = list(set(feas)-(set(loss_drop)|set(unique_drop)|set(iv_drop)|set(psi_drop)|set(psi_bad_drop)|set(add_drop_fes)|set(corr_drop)))
len(feas1)

10

In [328]:
feas1

['feat_loan_risk_province_from_idcard',
 'mayi_hello_v6',
 'blz_yaosi_ty_2',
 'zj_50002',
 'mayi_afc_yh_cus_dq_v2',
 'dxm_V12PRO-hjgdj2',
 'feat_loan_risk_gender_from_idcard',
 'feat_loan_risk_age_from_idcard',
 'xys_fc92',
 'xys_linglong92']

In [329]:
feas_round1 = feas1
len(feas_round1)

10

In [330]:
# 创建成lgb特征的数据集格式
round_fes = feas_round1.copy()

lgb_train = lgb.Dataset(train[round_fes], y_train, categorical_feature=cat_fes)
lgb_eval = lgb.Dataset(test[round_fes], y_test, categorical_feature=cat_fes, reference=lgb_train)

params_dict = {
    
# 'n_estimators': 150, 'learning_rate': 0.25, 'num_leaves': 6, 'max_depth': 4, 'min_data_in_leaf': 245, 'lambda_l1': 26, 'lambda_l2': 58, 'min_split_gain': 7.0, 'subsample': 0.8, 'colsample_bytree': 1.0, 'max_bin': 9
# 'n_estimators': 150, 'learning_rate': 0.060000000000000005, 'num_leaves': 4, 'max_depth': 4, 'min_data_in_leaf': 275, 'lambda_l1': 58, 'lambda_l2': 100, 'min_split_gain': 10.0, 'subsample': 0.9, 'colsample_bytree': 1.0, 'max_bin': 9


# 'n_estimators': 150, 'learning_rate': 0.060000000000000005, 'num_leaves': 11, 'max_depth': 3, 'min_data_in_leaf': 170, 'lambda_l1': 76, 'lambda_l2': 82, 'min_split_gain': 10.0, 'subsample': 0.8, 'colsample_bytree': 1.0, 'max_bin': 10
# 'n_estimators': 150, 'learning_rate': 0.09, 'num_leaves': 6, 'max_depth': 3, 'min_data_in_leaf': 130, 'lambda_l1': 58, 'lambda_l2': 86, 'min_split_gain': 9.0, 'subsample': 0.9, 'colsample_bytree': 1.0, 'max_bin': 10
# 'n_estimators': 150, 'learning_rate': 0.060000000000000005, 'num_leaves': 11, 'max_depth': 3, 'min_data_in_leaf': 100, 'lambda_l1': 38, 'lambda_l2': 78, 'min_split_gain': 10.0, 'subsample': 0.9, 'colsample_bytree': 1.0, 'max_bin': 10

# 'n_estimators': 150, 'learning_rate': 0.11, 'num_leaves': 5, 'max_depth': 4, 'min_data_in_leaf': 195, 'lambda_l1': 68, 'lambda_l2': 88, 'min_split_gain': 6.0, 'subsample': 0.9, 'colsample_bytree': 1.0, 'max_bin': 10
# 'n_estimators': 150, 'learning_rate': 0.07, 'num_leaves': 7, 'max_depth': 3, 'min_data_in_leaf': 250, 'lambda_l1': 62, 'lambda_l2': 78, 'min_split_gain': 8.0, 'subsample': 0.9, 'colsample_bytree': 1.0, 'max_bin': 9
'n_estimators': 150, 'learning_rate': 0.1, 'num_leaves': 5, 'max_depth': 4, 'min_data_in_leaf': 210, 'lambda_l1': 72, 'lambda_l2': 72, 'min_split_gain': 7.0, 'subsample': 0.9, 'colsample_bytree': 1.0, 'max_bin': 10

}


# 将参数写成字典下形式
adj_params = {
    'boosting_type':'gbdt',
    # 'class_weight':None,
    'objective':'binary',
    'metrics':'auc',
    'learning_rate': params_dict['learning_rate'],
    'max_depth': params_dict['max_depth'],
    'num_leaves':params_dict['num_leaves'],  # 叶子节点数
    'max_bin': params_dict['max_bin'],
    'subsample':params_dict['subsample'],
    'colsample_bytree':params_dict['colsample_bytree'], #特征采样比例，默认值为1，推荐的候选值为：[0.6, 0.7, 0.8, 0.9, 1]
    'lambda_l1':params_dict['lambda_l1'],    #l1
    'lambda_l2': params_dict['lambda_l2'],  #l2
#     'min_child_weight':0.02,    #一个叶子上的最小hessian和
    'min_split_gain':params_dict['min_split_gain'],
    'min_data_in_leaf': params_dict['min_data_in_leaf'],  #一个叶子上的最小数据量。默认设置为20。根据数据量来确定，
    'n_estimators': params_dict['n_estimators'],   #boosting的迭代次数。默认设置为100。一般根据数据集和特征数据选择100~1000之间。
#     'subsample_freq': 0,
    # 'scale_pos_weight':2,
    'bagging_freq':5,
    'is_unbalance':True,
    'random_state':123,   #随机种子
    'verbose': -1,
    'nthread':4 #线程数
}

#训练模型,verbose_eval间隔20次输出一次信息
lgbm = lgb.train(adj_params, lgb_train, valid_sets=[lgb_train, lgb_eval], early_stopping_rounds=20)

[1]	training's auc: 0.593577	valid_1's auc: 0.592628
Training until validation scores don't improve for 20 rounds
[2]	training's auc: 0.626003	valid_1's auc: 0.62361
[3]	training's auc: 0.634573	valid_1's auc: 0.636112
[4]	training's auc: 0.637741	valid_1's auc: 0.637435
[5]	training's auc: 0.638909	valid_1's auc: 0.636518
[6]	training's auc: 0.64121	valid_1's auc: 0.636895
[7]	training's auc: 0.641396	valid_1's auc: 0.63727
[8]	training's auc: 0.642753	valid_1's auc: 0.637974
[9]	training's auc: 0.643426	valid_1's auc: 0.639197
[10]	training's auc: 0.644749	valid_1's auc: 0.640392
[11]	training's auc: 0.645677	valid_1's auc: 0.641094
[12]	training's auc: 0.647069	valid_1's auc: 0.6424
[13]	training's auc: 0.648124	valid_1's auc: 0.643775
[14]	training's auc: 0.648837	valid_1's auc: 0.644853
[15]	training's auc: 0.649504	valid_1's auc: 0.644708
[16]	training's auc: 0.650339	valid_1's auc: 0.645357
[17]	training's auc: 0.650993	valid_1's auc: 0.646047
[18]	training's auc: 0.651565	valid

In [331]:
pred_train = lgbm.predict(train[round_fes])
pred_test = lgbm.predict(test[round_fes])
pred_oot = lgbm.predict(df_oot[round_fes])
pred_oot1 = lgbm.predict(df_oot1[round_fes])


train_res = mf.binary_metric(y_train,pred_train)
test_res = mf.binary_metric(y_test,pred_test)
oot_res = mf.binary_metric(y_oot,pred_oot)
oot_res1 = mf.binary_metric(y_oot1,pred_oot1)

 ------------------------- 
AUC:            66.88%
KS:             24.72%
------------------------- 
 ------------------------- 
AUC:            65.95%
KS:             25.03%
------------------------- 
 ------------------------- 
AUC:            66.19%
KS:             26.15%
------------------------- 
 ------------------------- 
AUC:            65.32%
KS:             24.88%
------------------------- 


In [332]:
pred_res = pd.DataFrame({'pred':pred_oot1, 'y':df_oot1[y_label]})
mf.qcut_lift_ks(pred_res, 'pred', 'y', special_list=[], max_bins=20, sort=True, precision=2)

,Event,NEvent,all,EventRate,Event_CumRate,NEvent_CumRate,all_CumRate,all_Rate,ks,lift,cum_lift
"(0.68, 0.79]",118.0,1273.0,1391.0,0.084831,0.121274,0.047427,0.050011,0.050011,0.073847,2.424965,2.424965
"(0.65, 0.68]",82.0,1309.0,1391.0,0.058950,0.205550,0.096196,0.100022,0.050011,0.109354,1.685145,2.055055
"(0.63, 0.65]",74.0,1316.0,1390.0,0.053237,0.281603,0.145226,0.149996,0.049975,0.136378,1.521835,1.877400
"(0.61, 0.63]",72.0,1319.0,1391.0,0.051761,0.355601,0.194367,0.200007,0.050011,0.161234,1.479640,1.777942
"(0.59, 0.61]",67.0,1324.0,1391.0,0.048167,0.424460,0.243694,0.250018,0.050011,0.180766,1.376887,1.697720
"(0.57, 0.59]",63.0,1327.0,1390.0,0.045324,0.489209,0.293134,0.299993,0.049975,0.196075,1.295616,1.630735
"(0.56, 0.57]",63.0,1328.0,1391.0,0.045291,0.553957,0.342610,0.350004,0.050011,0.211347,1.294685,1.582718
"(0.54, 0.56]",76.0,1315.0,1391.0,0.054637,0.632066,0.391602,0.400014,0.050011,0.240463,1.561842,1.580108
"(0.52, 0.54]",47.0,1343.0,1390.0,0.033813,0.680370,0.441638,0.449989,0.049975,0.238732,0.966571,1.511970
"(0.5, 0.52]",42.0,1349.0,1391.0,0.030194,0.723535,0.491897,0.500000,0.050011,0.231639,0.863123,1.447071


In [333]:
imp_df = pd.Series(lgbm.feature_importance(importance_type='gain'), index = round_fes,).sort_values(ascending=False)
imp_drop = imp_df[imp_df==0].index.tolist()
len(imp_drop)

0

In [334]:
imp_df

dxm_V12PRO-hjgdj2                      48968.799978
mayi_afc_yh_cus_dq_v2                  13833.699773
zj_50002                               11691.487173
xys_fc92                                9786.587025
blz_yaosi_ty_2                          8865.667516
mayi_hello_v6                           7891.153564
xys_linglong92                          5299.975226
feat_loan_risk_gender_from_idcard       2819.140804
feat_loan_risk_age_from_idcard          2587.902104
feat_loan_risk_province_from_idcard     2348.511808
dtype: float64

In [335]:
df_merge2 = df_merge.copy()
# df_merge2 = df_merge[df_merge.is_zy==1].copy()
# df_merge2 = df_merge[df_merge.is_zy==0].copy()
df_merge2[label_list] = df_merge2[label_list].replace(0.5, 0)
df_merge2.shape

(300000, 69)

In [336]:
model_scores = ['model_pred']
if_sort = True
parts_col = 'mon'
parts_col
df_merge2['model_pred'] = lgbm.predict(df_merge2[round_fes])
kq_tmp = ['aaa']
kq_list = {'aaa':df_merge2}

##分月性能评估
mon_info_all = {}
for score_one in tqdm(model_scores):
    model_pred = 'model_pred'
    model_score = 'model_pred'

    mon_info_score_one = {}

    for kq_one in kq_tmp:
        mon_info_kq_one = {}
        score_kq_one = kq_list[kq_one]
        #所有y标测算
        need_cols = list(set([parts_col, model_pred, model_score]+label_list))
        all_y_name = label_list
        all_score_df = score_kq_one[need_cols]

        parts = np.sort(all_score_df[parts_col].unique())
        part_name = np.sort(all_score_df[parts_col].unique())


        ks_all_df = pd.DataFrame()
        t5p_lift_all = pd.DataFrame()
        t10p_lift_all = pd.DataFrame()
        h10p_lift_all = pd.DataFrame()
        h5p_lift_all = pd.DataFrame()
        t2p_lift_all = pd.DataFrame()
        qcut_ks_lift_all = {}
        for i, part in enumerate(parts):
            ks_part = {}
            t5p_lift_part = {}
            t10p_lift_part = {}
            h10p_lift_part = {}
            h5p_lift_part = {}
            t2p_lift_part = {}
            qcut_ks_lift_part = {}
            for y_one in all_y_name:
                try:
                    df_temp = all_score_df[(all_score_df[parts_col].isin([part])) & (all_score_df[y_one].notnull())]
                    model_score_report_one = mf.model_score_report(df_temp, model_score, y_one, auc_score=model_pred, max_bins=10, precision=0, sort=if_sort, self_bins=[],
                                                                    lowerlift_per=[])
                    ks_part[y_one] = model_score_report_one['KS']
                    t5p_lift_part[y_one] = model_score_report_one['TH_lift']['尾部5%lift']
                    t10p_lift_part[y_one] = model_score_report_one['TH_lift']['尾部10%lift']
                    h10p_lift_part[y_one] = model_score_report_one['TH_lift']['头部10%lift']
                    h5p_lift_part[y_one] = model_score_report_one['TH_lift']['头部5%lift']
                    # t2p_lift_part[y_one] = model_score_report_one['lower_lift']['小于0.02lift']
                    qcut_ks_lift_part[y_one] = model_score_report_one['qcut_ks_lift']
                except:
                    ks_part[y_one] = np.nan
                    t5p_lift_part[y_one] = np.nan
                    t10p_lift_part[y_one] = np.nan
                    h10p_lift_part[y_one] = np.nan
                    h5p_lift_part[y_one] = np.nan
            
            ks_all_df[part_name[i]] = pd.Series(ks_part)
            t5p_lift_all[part_name[i]] = pd.Series(t5p_lift_part)
            t10p_lift_all[part_name[i]] = pd.Series(t10p_lift_part)
            h10p_lift_all[part_name[i]] = pd.Series(h10p_lift_part)
            h5p_lift_all[part_name[i]] = pd.Series(h5p_lift_part)
            t2p_lift_all[part_name[i]] = pd.Series(t2p_lift_part)
            qcut_ks_lift_all[part_name[i]] = qcut_ks_lift_part
        mon_info_kq_one['KS'] = ks_all_df.T
        mon_info_kq_one['尾部5%lift'] = t5p_lift_all.T
        mon_info_kq_one['尾部10%lift'] = t10p_lift_all.T
        mon_info_kq_one['头部10%lift'] = h10p_lift_all.T
        mon_info_kq_one['头部5%lift'] = h5p_lift_all.T
        mon_info_kq_one['尾部2%lift'] = t2p_lift_all.T
        mon_info_score_one[kq_one] = mon_info_kq_one
    mon_info_all[score_one] = mon_info_score_one

100%|██████████| 1/1 [00:02<00:00,  2.60s/it]


In [337]:
mon_info_all['model_pred']['aaa']['KS']

,label_fpd_7,label_fpd_15,label_fpd_30,label_2pd_7,label_2pd_15,label_2pd_30,label_3pd_7,label_3pd_15,label_3pd_30,label_4pd_15,label_4pd_30,label_5pd_30,label_6pd_30
2024-10,0.217066,0.212912,0.229950,0.227365,0.238484,0.248158,0.212764,0.220402,0.224711,0.205413,0.206674,0.198421,0.179113
2024-11,0.219994,0.207596,0.215719,0.218237,0.221395,0.216845,0.211973,0.212940,0.212055,0.214663,0.211923,0.191089,0.193727
2024-12,0.225548,0.240708,0.237562,0.233546,0.246321,0.237021,0.216723,0.215772,0.209774,0.201054,0.203188,0.205005,NaN
2025-01,0.255363,0.256193,0.276679,0.246519,0.247166,0.255374,0.230203,0.233431,0.235613,0.208764,0.231156,NaN,NaN
2025-02,0.224124,0.230301,0.227494,0.228960,0.221967,0.225177,0.223032,0.219667,0.237413,0.224387,NaN,NaN,NaN
2025-03,0.270239,0.267490,0.285210,0.243090,0.252444,0.261044,0.239235,0.301039,NaN,NaN,NaN,NaN,NaN
2025-04,0.232530,0.236312,0.276879,0.300123,0.411939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05,0.216894,0.165427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [338]:
mon_info_all['model_pred']['aaa']['尾部5%lift']

,label_fpd_7,label_fpd_15,label_fpd_30,label_2pd_7,label_2pd_15,label_2pd_30,label_3pd_7,label_3pd_15,label_3pd_30,label_4pd_15,label_4pd_30,label_5pd_30,label_6pd_30
2024-10,2.255087,2.184941,1.884788,2.015417,1.907152,1.886552,1.977378,1.954546,1.932155,1.997130,1.993393,2.025326,1.972545
2024-11,2.351837,2.234246,2.427432,2.077947,2.024366,2.104276,2.018757,2.147033,2.161148,2.135657,2.111181,1.970901,1.840560
2024-12,2.461573,2.833062,2.959769,2.414752,2.693725,2.642715,2.184795,2.298262,2.275964,2.110203,2.063426,2.198461,NaN
2025-01,2.903970,2.953766,2.962636,2.513535,2.561767,2.540853,2.339794,2.380689,2.369350,2.262460,2.728358,NaN,NaN
2025-02,2.209454,2.204098,2.165646,2.176096,2.154775,2.088811,2.128670,2.040773,2.097023,1.685671,NaN,NaN,NaN
2025-03,2.812440,2.940751,3.223343,2.463640,2.534293,2.737607,2.231156,2.454680,NaN,NaN,NaN,NaN,NaN
2025-04,2.696260,2.922755,3.408447,2.702703,4.252874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05,2.416667,1.965957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [339]:
mon_info_all['model_pred']['aaa']['尾部10%lift']

,label_fpd_7,label_fpd_15,label_fpd_30,label_2pd_7,label_2pd_15,label_2pd_30,label_3pd_7,label_3pd_15,label_3pd_30,label_4pd_15,label_4pd_30,label_5pd_30,label_6pd_30
2024-10,1.930304,1.884971,1.817916,1.997908,2.039176,2.057367,1.972110,2.003897,2.012486,1.929635,1.918087,1.886262,1.824964
2024-11,1.976089,1.852583,1.999614,1.972670,1.955894,1.956437,1.902342,2.009771,1.997976,2.033230,1.998420,1.905824,1.813885
2024-12,2.116631,2.288828,2.368421,2.204941,2.354874,2.344140,2.025172,2.075727,2.047364,1.902508,1.874889,1.958005,NaN
2025-01,2.210977,2.215324,2.268268,2.188985,2.177502,2.193542,2.063751,2.086902,2.083006,1.958963,2.182686,NaN,NaN
2025-02,2.120823,2.099698,2.078153,2.056452,2.010582,1.998404,1.977568,1.946396,1.957967,1.609050,NaN,NaN,NaN
2025-03,2.280955,2.299133,2.449390,2.097684,2.157253,2.289825,2.009448,2.743466,NaN,NaN,NaN,NaN,NaN
2025-04,2.359227,2.358132,2.822398,2.612613,3.402299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05,1.816555,1.987097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [340]:
mon_info_all['model_pred']['aaa']['头部10%lift']

,label_fpd_7,label_fpd_15,label_fpd_30,label_2pd_7,label_2pd_15,label_2pd_30,label_3pd_7,label_3pd_15,label_3pd_30,label_4pd_15,label_4pd_30,label_5pd_30,label_6pd_30
2024-10,0.195199,0.245866,0.235656,0.233989,0.230230,0.222772,0.224234,0.230413,0.247568,0.303303,0.303163,0.351142,0.374352
2024-11,0.282298,0.264655,0.249952,0.336797,0.345158,0.364302,0.350917,0.326497,0.302972,0.330739,0.328099,0.376516,0.420128
2024-12,0.215983,0.163488,0.131579,0.256176,0.208105,0.199501,0.280320,0.277214,0.267380,0.326550,0.340413,0.395036,NaN
2025-01,0.280497,0.299368,0.277747,0.241686,0.240166,0.255913,0.280425,0.288722,0.275114,0.346378,0.314175,NaN,NaN
2025-02,0.269923,0.241692,0.230906,0.299539,0.291005,0.272510,0.321586,0.307088,0.297191,0.229864,NaN,NaN,NaN
2025-03,0.287888,0.240607,0.232537,0.316027,0.269606,0.230133,0.334908,0.288786,NaN,NaN,NaN,NaN,NaN
2025-04,0.280860,0.265705,0.117600,0.300300,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05,0.151380,0.993548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [341]:
mon_info_all['model_pred']['aaa']['头部5%lift']

,label_fpd_7,label_fpd_15,label_fpd_30,label_2pd_7,label_2pd_15,label_2pd_30,label_3pd_7,label_3pd_15,label_3pd_30,label_4pd_15,label_4pd_30,label_5pd_30,label_6pd_30
2024-10,0.216835,0.273118,0.269255,0.143958,0.153449,0.157213,0.149453,0.139610,0.159682,0.230036,0.256460,0.279045,0.316759
2024-11,0.376294,0.352776,0.285580,0.404045,0.437079,0.458624,0.283118,0.290140,0.294702,0.303545,0.298189,0.362571,0.386784
2024-12,0.215927,0.163446,0.131545,0.274404,0.240902,0.224381,0.274529,0.270384,0.259674,0.293347,0.303753,0.326334,NaN
2025-01,0.362996,0.359242,0.324038,0.290023,0.288199,0.310752,0.350531,0.354571,0.325645,0.377077,0.413388,NaN,NaN
2025-02,0.257017,0.241643,0.213101,0.241886,0.211597,0.211908,0.251301,0.261869,0.262128,0.153243,NaN,NaN,NaN
2025-03,0.265742,0.240607,0.217035,0.247280,0.226469,0.207046,0.283965,0.288786,NaN,NaN,NaN,NaN,NaN
2025-04,0.112267,0.132758,0.000000,0.180180,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [342]:
# df_merge['mon'] = df_merge['date'].str[:7]
df_TT_merge1 = df_merge.copy()
df_TT_merge1['model_pred'] = lgbm.predict(df_TT_merge1[round_fes])

In [343]:
#模型分稳定性
parts_col = 'mon'
model_score = 'model_pred'
psi_pred_df = df_TT_merge1[[parts_col, model_score]]
mons = np.sort(psi_pred_df[parts_col].unique())
psi_res = {}
psi_base = {}
psi_df_model = pd.DataFrame()
base_score = df_TT_merge1[(df_TT_merge1.mon<='2024-12') & (df_TT_merge1.mon>='2024-10')][model_score]
for i, mon in enumerate(mons):
    base_df = psi_pred_df[psi_pred_df[parts_col]==mon][model_score]
    psi_base[mon], _, _ = mf.calculate_psi(base_score, base_df, bins=10, min_sample=10, numerical=True)
    if mon<max(mons):
        mon_n = mons[i+1]
        test_df = psi_pred_df[psi_pred_df[parts_col]==mon_n][model_score]
        psi_res[mon_n], _, _ = mf.calculate_psi(base_df, test_df, bins=10, min_sample=10, numerical=True)
psi_df_model['train为基期'] = pd.Series(psi_base)
psi_df_model['月环比'] = pd.Series(psi_res)
psi_df_model

,train为基期,月环比
2024-10,0.000143,NaN
2024-11,0.001739,0.002173
2024-12,0.001489,0.006518
2025-01,0.005813,0.001715
2025-02,0.017906,0.003676
2025-03,0.035663,0.003265
2025-04,0.010799,0.023789
2025-05,0.038323,0.015937


In [345]:
joblib.dump(lgbm, 'lgbm_uplevel_v1_0.pkl')

['lgbm_uplevel_v1_0.pkl']

In [ ]:
# df_oot_tmp = df_merge2[(df_merge2.mon=='2025-03')&(df_merge2.mob1_15.notnull())]
# df_oot_tmp.shape

In [41]:
import optuna
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.model_selection import StratifiedKFold

def objective(trial, x_train_obj, y_train_obj, x_test_obj, y_test_obj, x_oot_obj, y_oot_obj, x_oot1_obj, y_oot1_obj, fea_cat_obj):
    lgb_train = lgb.Dataset(x_train_obj, y_train_obj, categorical_feature=fea_cat_obj)
    lgb_eval = lgb.Dataset(x_test_obj, y_test_obj, categorical_feature=fea_cat_obj, reference=lgb_train)
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 150,150, step=1),
        "learning_rate": trial.suggest_float("learning_rate", 0.05, 0.15, step=0.01),
        "num_leaves": trial.suggest_int("num_leaves", 4,12, step=1),
        "max_depth": trial.suggest_int("max_depth", 3,4, step=1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 100, 290, step=5),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=2),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=2),
        "min_split_gain": trial.suggest_float("min_split_gain", 5,10, step=1),
#         'min_child_weight': trial.suggest_float("min_child_weight", 0, 1, step=0.1),
        "subsample": trial.suggest_float("subsample", 0.8, 0.9, step=0.1),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 1, 1, step=0.1),
        'max_bin': trial.suggest_int("max_bin", 6, 10, step=1),
        "random_state": 123,
        'metric': 'auc',
        'objective':"binary", 
        'boosting_type':'gbdt',  
        'bagging_freq':5,
        # 'scale_pos_weight':1,
        'is_unbalance':True,
        'verbose':-1,
        'nthread':4, #线程数
    }
    
    
    
    #训练模型,verbose_eval间隔20次输出一次信息
    lgbm = lgb.train(params, lgb_train, verbose_eval=False, valid_sets=[lgb_train, lgb_eval], early_stopping_rounds=20)
    test_pred=lgbm.predict(x_test_obj)
    train_pred=lgbm.predict(x_train_obj)
    oot_pred = lgbm.predict(x_oot_obj)
    oot1_pred = lgbm.predict(x_oot1_obj)
    
    fpr, tpr, thresholds = roc_curve(y_test_obj, test_pred, pos_label=1)   #roc_curve ROC曲线
    fpr1, tpr1, thresholds1 = roc_curve(y_train_obj, train_pred, pos_label=1)   #roc_curve ROC曲线
    fpr2, tpr2, thresholds2 = roc_curve(y_oot_obj, oot_pred, pos_label=1)   #roc_curve ROC曲线
    fpr3, tpr3, thresholds3 = roc_curve(y_oot1_obj, oot1_pred, pos_label=1)   #roc_curve ROC曲线
    auc = roc_auc_score(y_test_obj, test_pred)     #roc_auc_score  AUC面积
    KS_test = max(abs(tpr-fpr))
    KS_train = max(abs(tpr1-fpr1))
    KS_oot = max(abs(tpr2-fpr2))
    KS_oot1 = max(abs(tpr3-fpr3))
    diff_ks = abs(KS_train-KS_test)
    pred_res = pd.DataFrame({'pred':oot1_pred, 'y':y_oot1_obj})
    lift_tmp = mf.qcut_lift_ks(pred_res, 'pred', 'y', special_list=[], max_bins=20, sort=True, precision=2)['lift'][0]
    pred_res1 = pd.DataFrame({'pred':oot_pred, 'y':y_oot_obj})
    lift_tmp1 = mf.qcut_lift_ks(pred_res1, 'pred', 'y', special_list=[], max_bins=20, sort=True, precision=2)['lift'][0]

    print(KS_train, KS_test, KS_oot, KS_oot1, lift_tmp1, lift_tmp)

    return KS_test, diff_ks
    # return KS_oot1


In [42]:
fea_obj = feas_round1.copy()
fea_cat_obj = cat_fes
study = optuna.create_study(directions=["maximize", 'minimize'])
# study = optuna.create_study(directions=["maximize"])
func = lambda trial: objective(trial, train[fea_obj], y_train, test[fea_obj], y_test, df_oot[fea_obj], df_oot[y_label], df_oot1[fea_obj], df_oot1[y_label], fea_cat_obj)
study.optimize(func, n_trials=250)

[I 2025-06-19 13:24:18,441] A new study created in memory with name: no-name-51ebd5ce-58ac-419d-99a2-26936af909f2


[I 2025-06-19 13:24:19,722] Trial 0 finished with values: [0.24500190618804157, 0.0006083342528550029] and parameters: {'n_estimators': 150, 'learning_rate': 0.07, 'num_leaves': 6, 'max_depth': 4, 'min_data_in_leaf': 200, 'lambda_l1': 80, 'lambda_l2': 40, 'min_split_gain': 8.0, 'subsample': 0.9, 'colsample_bytree': 1.0, 'max_bin': 10}. 


0.24561024044089658 0.24500190618804157 0.2521523189968233 0.2385646768475143 2.4443003853413496 2.383863967673556


[I 2025-06-19 13:24:20,532] Trial 1 finished with values: [0.24898412101682316, 0.003154610551082959] and parameters: {'n_estimators': 150, 'learning_rate': 0.12000000000000001, 'num_leaves': 9, 'max_depth': 4, 'min_data_in_leaf': 225, 'lambda_l1': 82, 'lambda_l2': 82, 'min_split_gain': 9.0, 'subsample': 0.9, 'colsample_bytree': 1.0, 'max_bin': 10}. 


0.2521387315679061 0.24898412101682316 0.2579540465322789 0.24561226970458633 2.4032197065961167 2.342762864782632


...

0.2488254649419142 0.2503721769431069 0.25049075362734363 0.23774503525442914 2.362139027850884 2.2605606590007854


[I 2025-06-19 13:28:28,474] Trial 248 finished with values: [0.24623189773548543, 0.007166187871191998] and parameters: {'n_estimators': 150, 'learning_rate': 0.11, 'num_leaves': 11, 'max_depth': 3, 'min_data_in_leaf': 165, 'lambda_l1': 58, 'lambda_l2': 92, 'min_split_gain': 5.0, 'subsample': 0.9, 'colsample_bytree': 1.0, 'max_bin': 10}. 


0.2533980856066774 0.24623189773548543 0.26245592070287804 0.2494867858926227 2.4853810640865825 2.2605606590007854


[I 2025-06-19 13:28:29,784] Trial 249 finished with values: [0.23678813582006092, 0.0037130288852870175] and parameters: {'n_estimators': 150, 'learning_rate': 0.060000000000000005, 'num_leaves': 7, 'max_depth': 3, 'min_data_in_leaf': 285, 'lambda_l1': 24, 'lambda_l2': 24, 'min_split_gain': 8.0, 'subsample': 0.8, 'colsample_bytree': 1.0, 'max_bin': 6}. 


0.24050116470534794 0.23678813582006092 0.24814719777679017 0.23490228877429126 2.2202176897025043 2.034504593100707
